In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../src"))

from recommender.riot_api_helper import RiotApiHelper
from recommender.data_loaders.summoner_mastery_loader import SummonerMasteryLoader

In [5]:
import os
sml = SummonerMasteryLoader()

pkl_folder_path = sml.json_folder_path

combined_puuid_dict = {}
for pkl_file in os.listdir(pkl_folder_path):
    puuid = pkl_file.split(".pkl")[0]
    s = sml.load_dict_from_pkl(puuid)

In [6]:
t = map_helper.get_id_champ_map()

In [15]:
for key in s: 
    champ_id_str, suffix = key.split("_", 1)

In [1]:
from data_processors.summoner_mastery_processor import SummonerMasteryProcessor
smp = SummonerMasteryProcessor()

ImportError: attempted relative import beyond top-level package

In [2]:
smp.load_rating(True)

Loading new ratings...


,Aatrox,Ahri,Akali,Akshan,Alistar,Ambessa,Amumu,Anivia,Annie,Aphelios,...,Yone,Yorick,Yuumi,Zac,Zed,Zeri,Ziggs,Zilean,Zoe,Zyra
JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfAoYd9LVuHP2b_CckQhhcVpcvo6XNx184A,3.702635,7.020581,2.340194,1.170097,8.758095,2.096557,2.340194,2.340194,2.340194,1.170097,...,3.475027,8.501635,9.104315,117.573892,2.442777,1.170097,4.635507,4.635507,1.170097,6.67436


In [3]:
from data_loaders.summoner_mastery_loader import SummonerMasteryLoader
sml = SummonerMasteryLoader()

ImportError: attempted relative import beyond top-level package

In [3]:
a = smp.aggregate_summoner_pkls()

In [7]:
a["JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfAoYd9LVuHP2b_CckQhhcVpcvo6XNx184A"].keys()


dict_keys(['Zac_level', 'Zac_points', 'Zac_last_play_time', 'Rengar_level', 'Rengar_points', 'Rengar_last_play_time', 'RekSai_level', 'RekSai_points', 'RekSai_last_play_time', 'Udyr_level', 'Udyr_points', 'Udyr_last_play_time', 'Maokai_level', 'Maokai_points', 'Maokai_last_play_time', 'Nautilus_level', 'Nautilus_points', 'Nautilus_last_play_time', 'Jhin_level', 'Jhin_points', 'Jhin_last_play_time', 'Thresh_level', 'Thresh_points', 'Thresh_last_play_time', 'Ezreal_level', 'Ezreal_points', 'Ezreal_last_play_time', 'Volibear_level', 'Volibear_points', 'Volibear_last_play_time', 'Brand_level', 'Brand_points', 'Brand_last_play_time', 'Illaoi_level', 'Illaoi_points', 'Illaoi_last_play_time', 'Gragas_level', 'Gragas_points', 'Gragas_last_play_time', 'LeeSin_level', 'LeeSin_points', 'LeeSin_last_play_time', 'Sejuani_level', 'Sejuani_points', 'Sejuani_last_play_time', 'Lulu_level', 'Lulu_points', 'Lulu_last_play_time', 'Hecarim_level', 'Hecarim_points', 'Hecarim_last_play_time', 'Smolder_level'

In [8]:
from summoner_mastery_loader import SummonerMasteryLoader
sml = SummonerMasteryLoader()
t = sml.load_dict_from_pkl(puuid)

In [ ]:
for key

{'154_level': 53,
 '154_points': 551071,
 '154_last_play_time': 1745906806000,
 '107_level': 22,
 '107_points': 245718,
 '107_last_play_time': 1710812395000,
 '421_level': 15,
 '421_points': 174276,
 '421_last_play_time': 1636932241000,
 '77_level': 6,
 '77_points': 35918,
 '77_last_play_time': 1734734695000,
 '57_level': 6,
 '57_points': 33615,
 '57_last_play_time': 1707001407000,
 '111_level': 6,
 '111_points': 32861,
 '111_last_play_time': 1742533806000,
 '202_level': 5,
 '202_points': 28071,
 '202_last_play_time': 1731557505000,
 '412_level': 5,
 '412_points': 27089,
 '412_last_play_time': 1747514923000,
 '81_level': 5,
 '81_points': 26738,
 '81_last_play_time': 1748747376000,
 '106_level': 6,
 '106_points': 25588,
 '106_last_play_time': 1747518727000,
 '63_level': 5,
 '63_points': 24879,
 '63_last_play_time': 1744343167000,
 '420_level': 5,
 '420_points': 24115,
 '420_last_play_time': 1734055826000,
 '79_level': 5,
 '79_points': 22228,
 '79_last_play_time': 1747520679000,
 '64_lev

In [3]:
df, le_user, le_champion = summoner_data_processor.load_encoded_ratings()

# Train a model
train_df, test_df = train_test_split(df, test_size=0.2)
train_dataset = ChampionsDataset(train_df)
test_dataset = ChampionsDataset(test_df)
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=True)

num_summoners = len(df["puuid"].unique())
num_champions = len(df["champ_id"].unique())
num_factors = 10

model = DotProduct(num_summoners, num_champions, num_factors)
train_and_evaluate_model(model, train_loader, test_loader, epochs=25)

Epoch 1, Loss: 9.787265053468085
Test MSE: 9.503943857238678
Epoch 2, Loss: 8.955349342039346
Test MSE: 8.744121758471476
Epoch 3, Loss: 8.218760348012498
Test MSE: 8.06734226217534
Epoch 4, Loss: 7.56448665752106
Test MSE: 7.461733413545366
Epoch 5, Loss: 6.979558458462451
Test MSE: 6.917624471630774
Epoch 6, Loss: 6.455552323813899
Test MSE: 6.426868134031982
Epoch 7, Loss: 5.984084916413037
Test MSE: 5.982667671734141
Epoch 8, Loss: 5.557738807608967
Test MSE: 5.579331009166525
Epoch 9, Loss: 5.172148396647415
Test MSE: 5.211952502180557
Epoch 10, Loss: 4.8208994032857815
Test MSE: 4.876445989683885
Epoch 11, Loss: 4.501081686329726
Test MSE: 4.569227946511931
Epoch 12, Loss: 4.208590460080418
Test MSE: 4.28724769903584
Epoch 13, Loss: 3.9410002705452754
Test MSE: 4.02782102773622
Epoch 14, Loss: 3.6949742820380544
Test MSE: 3.788657081477381
Epoch 15, Loss: 3.4685206389452037
Test MSE: 3.5677460739439444
Epoch 16, Loss: 3.2600811604330158
Test MSE: 3.3632858268594266
Epoch 17, Loss

In [4]:
user_idx = le_user.transform([puuid])

# Get preds
all_champions = torch.tensor([champ_idx for champ_idx in range(num_champions)])
user_id = torch.tensor(list(user_idx) * len(all_champions))
predicted_ratings = model(user_id, all_champions)
_, top_indices = torch.topk(predicted_ratings, k=3)
top_indices_list = top_indices.tolist()

In [5]:
champ_order = [x for x in le_champion.inverse_transform(range(num_champions))]
for idx in top_indices_list:
    print(f"WE PRESCRIBE THAT YOU PLAY: {champ_order[idx]}")

WE PRESCRIBE THAT YOU PLAY: Varus
WE PRESCRIBE THAT YOU PLAY: Lillia
WE PRESCRIBE THAT YOU PLAY: Syndra


In [6]:
puuid = map_helper.get_puuid_mapping("BSIZZLEMONEY#0000")
puuid

'JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfAoYd9LVuHP2b_CckQhhcVpcvo6XNx184A'

In [10]:
df[df["puuid"] == puuid].sort_values("rating", ascending=False).head(20)

,puuid,champ_name,rating,user_id,champ_id
284,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,MissFortune,0.0,191,83
813,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Gwen,0.0,191,42
1342,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Jhin,0.0,191,53
1871,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Lux,0.0,191,76
2400,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Pyke,0.0,191,101
2929,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Caitlyn,0.0,191,20
3458,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Vayne,0.0,191,146
3987,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,TahmKench,0.0,191,132
4516,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Morgana,0.0,191,86
5045,JOvD6lK0bgPYCS-m-NtjRFn1bHZ_lf8zUQKMCBn-UZcUfA...,Ashe,0.0,191,10


In [8]:
sdl = SummonerDataLoader()
mine = sdl.load_dict_from_pkl(puuid)

In [9]:
mine

{'Zac': 24,
 'Zac_kills': 87,
 'Zac_assists': 223,
 'Zac_deaths': 93,
 'Zac_wins': 9,
 'Brand': 1,
 'Brand_kills': 3,
 'Brand_assists': 4,
 'Brand_deaths': 7}